In [ ]:
from pathlib import Path
PLOT_DIR = Path('plots')
PLOT_DIR.mkdir(exist_ok=True)
(PLOT_DIR / '.gitignore').write_text('*')

In [ ]:
%pip install seaborn

In [ ]:
import seaborn as sns
import seaborn.objects as so
from seaborn import axes_style, move_legend
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
names=["_1", "_2", "_3", "_4", "K", "Degree", "Probability"]
df = pd.concat([pd.read_csv(f, names=names) for f in glob("data/saved/degree-*.csv")], ignore_index=True)
df

In [ ]:
p = (
    so.Plot(
        data=df[df["Degree"] <= 8000],
        # data=df,
        x="Degree", y="Probability",
        color="K",
    )
    .theme({**axes_style("ticks")})
    .scale(
        y="log",
        color=so.Continuous().tick(at=df["K"].unique()),
    )
    .add(so.Line())
)
p

In [ ]:
for k in df["K"].unique():
    data = df[df["K"] == k]
    d = (data["Degree"] * data["Probability"]).sum()
    print(k, d)

In [ ]:
df = pd.concat([pd.read_csv(f, names=[
    "Number of Nodes",
    "Number of Data Objects",
    "Average Size of Failure Domains",
    "Protocol",
    "Redundancy",
    "Placement Group Size",
    "Step",
    "% Alive Data",
    "Average Node Failures per Step",
    "Churn Rate",
]) for f in glob("data/saved/*.csv")], ignore_index=True)
df["Average % of Failure Domains"] = df["Average Size of Failure Domains"] / df["Number of Nodes"] * 100
df

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
f.subplots_adjust(hspace=0.3)

In [ ]:
p = (
    so.Plot(
        data=df[(df["Average Size of Failure Domains"] != 0) & (df["Redundancy"] == 1.6) & (df["Placement Group Size"] == 8)], 
        x="Step", y="% Alive Data", color="Average % of Failure Domains")
    .theme({**axes_style("ticks")})
    .add(so.Line())
    .scale(
        color=so.Continuous().tick(at=[0.3, 0.5, 1, 2, 3]).label(like="{x}%"),
        x=so.Continuous().tick(at=[x * 2e5 for x in range(5 + 1)])
    )
    .label(color=None, x=None, y=None)
    .layout(size=(4, 3))
    .on(ax2)
    .plot()
)
legend = f.legends.pop(0)
ax2.legend(legend.legend_handles, [t.get_text() for t in legend.texts])
p

In [ ]:
p.save(PLOT_DIR / 'domain-scale.pdf', bbox_inches='tight')

In [ ]:
p = (
    so.Plot(
        data=df[(df["Average % of Failure Domains"] == 3) & ((df["Redundancy"] > 1.6) | (df["Placement Group Size"] == 8))], 
        x="Step", y="% Alive Data", color="Redundancy")
    .theme({**axes_style("ticks")})
    .add(so.Line())
    .scale(
        color=so.Continuous().tick(at=[x / 5 for x in range(8, 12)]).label(like="{x}x"),
        x=so.Continuous().tick(at=[x * 2e5 for x in range(5 + 1)])
    )
    .label(color=None, x=None, y=None)
    .layout(size=(4, 3))
    .on(ax3)
    .plot()
)
legend = f.legends.pop(0)
ax3.legend(legend.legend_handles, [t.get_text() for t in legend.texts])
p

In [ ]:
p.save(PLOT_DIR / 'redundancy.pdf', bbox_inches='tight')

In [ ]:
p = (
    so.Plot(
        data=df[(df["Average % of Failure Domains"] == 3) & ((df["Redundancy"] == 1.6))], 
        x="Step", y="% Alive Data", color="Placement Group Size")
    .theme({**axes_style("ticks")})
    .add(so.Line())
    .scale(
        color=so.Continuous().tick(at=list(range(8, 24 + 1, 8))),
        x=so.Continuous().tick(at=[x * 2e5 for x in range(5 + 1)])
    )
    .label(color=None, x=None, y=None)
    .layout(size=(4, 3))
    .on(ax4)
    .plot()
)
legend = f.legends.pop(0)
ax4.legend(legend.legend_handles, [t.get_text() for t in legend.texts])
p

In [ ]:
p.save(PLOT_DIR / 'group-scale.pdf', bbox_inches='tight')

In [ ]:
p = (
    so.Plot(
        data=df[df["Average Size of Failure Domains"] == 0], 
        x="Step", y="% Alive Data", color="Protocol")
    .theme({**axes_style("ticks")})
    .add(so.Line())
    # .scale(color=so.Continuous().tick(at=[x / 5 for x in range(8, 12)]))
    .scale(
        x=so.Continuous().tick(at=[x * 2e5 for x in range(5 + 1)])
    )
    .label(color=None, x=None, y=None)
    .layout(size=(4, 3))
    .on(ax1)
    .plot()
)
legend = f.legends.pop(0)
ax1.legend(legend.legend_handles, [t.get_text() for t in legend.texts])
p

In [ ]:
f.supxlabel('Step')
f.supylabel('% Alive Data')
f

In [ ]:
p.save(PLOT_DIR / 'repair.pdf', bbox_inches='tight')

In [ ]:
f.savefig(PLOT_DIR / 'all-in-one.pdf', bbox_inches='tight')